<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Copying-images-to-training,-validation,-and-test_directories" data-toc-modified-id="Copying-images-to-training,-validation,-and-test_directories-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Copying images to training, validation, and test_directories</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Data-Preprocessing" data-toc-modified-id="Data-Preprocessing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Preprocessing</a></span></li><li><span><a href="#Fitting-the-model-using-a-batch-generator" data-toc-modified-id="Fitting-the-model-using-a-batch-generator-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Fitting the model using a batch generator</a></span></li><li><span><a href="#Saving-the-model" data-toc-modified-id="Saving-the-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saving the model</a></span></li><li><span><a href="#Displaying-curves-of-loss-and-accuracy-during-training" data-toc-modified-id="Displaying-curves-of-loss-and-accuracy-during-training-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Displaying curves of loss and accuracy during training</a></span></li></ul></div>

# Load data

In [1]:
import os 
import shutil

In [2]:
original_dataset_dir = '/home/ubuntu/IEEE/train'
train_dir = '/home/ubuntu/IEEE/camera_model_small/train'
validation_dir = '/home/ubuntu/IEEE/camera_model_small/validation'
test_dir = '/home/ubuntu/IEEE/camera_model_small/test'

In [3]:
# Get camera list
cameras = os.listdir(original_dataset_dir)
cameras

['HTC-1-M7',
 'Motorola-X',
 'iPhone-6',
 'Motorola-Nexus-6',
 'Samsung-Galaxy-Note3',
 'iPhone-4s',
 'Motorola-Droid-Maxx',
 'Sony-NEX-7',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4']

In [7]:
for camera in cameras:
    camera_dir = os.path.join(original_dataset_dir, camera)
    print('Total training {} images: {} '.format(camera, len(os.listdir(camera_dir))))

Total training HTC-1-M7 images: 275 
Total training Motorola-X images: 275 
Total training iPhone-6 images: 275 
Total training Motorola-Nexus-6 images: 275 
Total training Samsung-Galaxy-Note3 images: 275 
Total training iPhone-4s images: 275 
Total training Motorola-Droid-Maxx images: 275 
Total training Sony-NEX-7 images: 275 
Total training LG-Nexus-5x images: 275 
Total training Samsung-Galaxy-S4 images: 275 


In [8]:
for camera in cameras:
    camera_dir = os.path.join(train_dir, camera)
    print('Total training {} images: {} '.format(camera, len(os.listdir(camera_dir))))


Total training HTC-1-M7 images: 165 
Total training Motorola-X images: 165 
Total training iPhone-6 images: 165 
Total training Motorola-Nexus-6 images: 165 
Total training Samsung-Galaxy-Note3 images: 165 
Total training iPhone-4s images: 165 
Total training Motorola-Droid-Maxx images: 165 
Total training Sony-NEX-7 images: 165 
Total training LG-Nexus-5x images: 165 
Total training Samsung-Galaxy-S4 images: 165 


In [9]:
for camera in cameras:
    camera_dir = os.path.join(validation_dir, camera)
    print('Total validation {} images: {} '.format(camera, len(os.listdir(camera_dir))))

Total validation HTC-1-M7 images: 55 
Total validation Motorola-X images: 55 
Total validation iPhone-6 images: 55 
Total validation Motorola-Nexus-6 images: 55 
Total validation Samsung-Galaxy-Note3 images: 55 
Total validation iPhone-4s images: 55 
Total validation Motorola-Droid-Maxx images: 55 
Total validation Sony-NEX-7 images: 55 
Total validation LG-Nexus-5x images: 55 
Total validation Samsung-Galaxy-S4 images: 55 


In [10]:
for camera in cameras:
    camera_dir = os.path.join(test_dir, camera)
    print('Total test {} images: {} '.format(camera, len(os.listdir(camera_dir))))

Total test HTC-1-M7 images: 55 
Total test Motorola-X images: 55 
Total test iPhone-6 images: 55 
Total test Motorola-Nexus-6 images: 55 
Total test Samsung-Galaxy-Note3 images: 55 
Total test iPhone-4s images: 55 
Total test Motorola-Droid-Maxx images: 55 
Total test Sony-NEX-7 images: 55 
Total test LG-Nexus-5x images: 55 
Total test Samsung-Galaxy-S4 images: 55 


# Using a pretrained convnet model

In [11]:
from keras import layers 
from keras import models
from keras.applications import InceptionResNetV2

Using TensorFlow backend.


In [12]:
def create_model(img_dim=(224, 224, 3)):
    input_tensor = layers.Input(shape=img_dim)
    conv_base = InceptionResNetV2(weights='imagenet',
                  include_top=False,
                  input_shape=img_dim)
    conv_base.trainable = True
    set_trainable = False
    for layer in conv_base.layers:
        if layer.name == 'block8_10_mixed':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
    
        else:
            layer.trainable = False
        
    bn = layers.normalization.BatchNormalization()(input_tensor)
    x = conv_base(bn)
    x = layers.Flatten()(x)
    output = layers.Dense(10, activation='softmax')(x)
    model = models.Model(input_tensor, output)
    return model

In [14]:
model = create_model()
model.summary()

219062272/219055592 [==============================] - 4s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
batch_normalization_204 (Bat (None, 224, 224, 3)       12        
_________________________________________________________________
inception_resnet_v2 (Model)  (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                384010    
Total params: 54,720,758
Trainable params: 4,514,352
Non-trainable params: 50,206,406
_________________________________________________________________


In [15]:
from keras import optimizers

In [16]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
             loss='categorical_crossentropy',
             metrics=['acc'])

# Data Preprocessing



Currently, the data sits on a drive as JPEG files, so the steps for getting it into the network are roughly as follows:

    Read the picture files
    Decode the JPEG content to RGB grids of pixels
    Convert these into floating point tensors
    Rescale the pixel values to the [0-1] range



In [17]:
from keras.preprocessing.image import ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 1650 images belonging to 10 classes.


In [20]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

Found 550 images belonging to 10 classes.


In [21]:
for data_batch, labels_batch in train_generator:
    print('data batch shape: ', data_batch.shape)
    print('labels batch shape: ', labels_batch.shape)
    break

data batch shape:  (16, 224, 224, 3)
labels batch shape:  (16, 10)


# Fitting the model using a batch generator

In [ ]:
%%time 
history = model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs = 30,
        validation_data = validation_generator,
        validation_steps = 50)


Epoch 1/30
 99/100 [============================>.] - ETA: 3s - loss: 2.2805 - acc: 0.2367

# Saving the model

In [ ]:
model.save('model_1.h5')

# Displaying curves of loss and accuracy during training

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline


In [ ]:
def plot_history(history):
    history_dict = history.history
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']
    acc_values = history_dict['acc']
    val_acc_values = history_dict['val_acc']
    epochs = range(1, len(acc_values) + 1)
    
    plt.plot(epochs, loss_values, 'b', label='Training loss')
    plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    plt.plot(epochs, acc_values, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc_values, 'r', label='Validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Validation')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)